In [1]:
from clickhouse_driver import Client
from mimesis import Person
from uuid import uuid4
import vertica_python


vertica_info = {
    'host': 'vertica',
    'port': 5433,
    'user': 'dbadmin',
    'password': 'dbpass',
    'database': 'docker',
    'autocommit': True,
} 

clickhouse_client = Client(host='clickhouse')
vertica_connection = vertica_python.connect(**vertica_info)
vertica_cursor = vertica_connection.cursor()
person = Person()

vertica_cursor.execute("""
CREATE TABLE if not exists test (
id UUID,
first_name VARCHAR,
age INT,
email VARCHAR,
political VARCHAR,
phone VARCHAR,
username VARCHAR
);

""")

clickhouse_client.execute("""
CREATE TABLE if not exists test (
id UUID,
first_name VARCHAR,
age INT,
email VARCHAR,
political VARCHAR,
phone VARCHAR,
username VARCHAR
) Engine=MergeTree() ORDER BY tuple();
""")

[]

In [2]:
import time
import statistics

class profile_code():
    def __init__(self):
        self.start = time.time()
        self.all_durations = []
        
    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.all_durations.append(end_time - self.start)
        self.start = end_time

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        print(f"Median - {statistics.median(self.all_durations)}")
        avg = sum(self.all_durations) / len(self.all_durations)
        print(f"Average - {avg}")
        print(f"Summary - {sum(self.all_durations)}")

## Генерация тестовых данных

In [3]:
test_data = [{'uuid': str(uuid4()),
            'first_name': person.name(), 
            'age': person.age(),
             'email': person.email(),
             'political': person.political_views(),
             'phone': person.telephone(),
             'username': person.username()
             } for x in range(10000)]

## Профилирование операций записи

**Vertica**

In [4]:
%%time
with profile_code() as profiler:
    for i in range(5):
        profiler.setup_start_time()
        for row in test_data:
            vertica_cursor.execute(f"""INSERT INTO test (id, first_name, age, email, political, phone, username) 
            VALUES ('{row['uuid']}', '{row['first_name']}', {row['age']}, '{row['email']}', 
            '{row['political']}', '{row['phone']}', '{row['username']}');""")
        profiler.checkpoint()
        vertica_cursor.execute("""TRUNCATE TABLE test;""")

Median - 28.983445644378662
Average - 29.051094484329223
Summary - 145.25547242164612
CPU times: user 3.26 s, sys: 986 ms, total: 4.24 s
Wall time: 2min 25s


**Clickhouse**

In [5]:
%%time
with profile_code() as profiler:
    for i in range(5):
        profiler.setup_start_time()
        for row in test_data:
            clickhouse_client.execute(f"""INSERT INTO test (id, first_name, age, email, political, phone, username) 
            VALUES ('{row['uuid']}', '{row['first_name']}', {row['age']}, '{row['email']}', 
            '{row['political']}', '{row['phone']}', '{row['username']}');""")
        profiler.checkpoint()
        clickhouse_client.execute("""TRUNCATE TABLE test;""")

Median - 8.567510604858398
Average - 8.514442682266235
Summary - 42.57221341133118
CPU times: user 5.41 s, sys: 2.84 s, total: 8.25 s
Wall time: 42.6 s


## Профилирование операций чтения

**Vertica**

In [6]:
%%time
with profile_code() as profiler:
    for i in range(10):
        profiler.setup_start_time()
        vertica_cursor.execute("""
        select * from test where age >= 10 AND political = 'Liberal' order by age;
        """)
        profiler.checkpoint()

Median - 0.003000974655151367
Average - 0.00697319507598877
Summary - 0.0697319507598877
CPU times: user 4.12 ms, sys: 1.37 ms, total: 5.49 ms
Wall time: 69.9 ms


**Clickhouse**

In [7]:
%%time
with profile_code() as profiler:
    for i in range(10):
        profiler.setup_start_time()
        clickhouse_client.execute("""
            select * from test where age >= 10 AND political = 'Liberal' order by age;
        """)
        profiler.checkpoint()

Median - 0.002900362014770508
Average - 0.004439353942871094
Summary - 0.04439353942871094
CPU times: user 10.5 ms, sys: 2.1 ms, total: 12.6 ms
Wall time: 44.6 ms


In [8]:
vertica_cursor.close()

**Исходя из полученных данных, видим что Clickhouse опережает Vertica по скорости чтения и записи, поэтому выбор Clickhouse в качестве аналитической БД более чем оправдан.**